# Exercise 1

First thing off, let's convert the MP3s to WAVs using the provided utilities:

In [1]:
from hw_utils import convert_all_mp3s

convert_all_mp3s()

  0%|          | 0/1413 [00:00<?, ?it/s]

Done!


In [1]:
# Extract the peaks

from tqdm.notebook import tqdm
from hw_utils import load_audio_peaks, tracks, OFFSET, DURATION, HOP_SIZE, N_TRACKS

peaks = {}

for track in tqdm(tracks, total=N_TRACKS):
    _, _, _, track_peaks = load_audio_peaks(track, OFFSET, DURATION, HOP_SIZE)
    peaks[str(track)] = track_peaks

  0%|          | 0/1413 [00:00<?, ?it/s]

In [3]:
# Save the extracted peaks

import pickle

with open("peaks.pkl", "wb") as peaks_file:
    pickle.dump(peaks, peaks_file)

Now that we've got our peaks, we convert them to multi-hot vectors.

So, rather than having [1,3,...] we'll convert to [0,1,0,1,...] so that set-based operations (like LSH/Jaccard) can be applied.

In [8]:
def to_multi_hot(dense_vector, length):
    result = [0]*length
    for i in dense_vector:
        if i < length:
            result[i] = 1
    return result

multihot_vectors_length = 0
for track_peaks in peaks.values():
    multihot_vectors_length = max(multihot_vectors_length, max(track_peaks if len(track_peaks) else [0]))
multihot_vectors_length += 1

print(f"Multi-hot vectors' length: {multihot_vectors_length}")

mh_peaks = {k: to_multi_hot(v, multihot_vectors_length) for k,v in peaks.items()}

431


To extract signatures from the vectors, we shuffle them multiple times, taking care of always using the same sequence of permutations

In [10]:
import random

SIGNATURES_LENGTH = 100

tracks_signatures = {}
for track_name, track_vector in mh_peaks.items():
    r = random.Random(123)  # Any seed value is fine as long as the same one is always used

    v = [value for value in track_vector]  # Copy the vector before starting shuffling
    track_signature = []
    for i in range(SIGNATURES_LENGTH):
        r.shuffle(v)
        # Append index of first non-0 value to the signature
        non_zeros = [i for i,value in enumerate(v) if value != 0]
        track_signature.append(non_zeros[0] if non_zeros else -1)

    tracks_signatures[track_name] = track_signature

with open("tracks_signatures.pkl", "wb") as pkl_out:
    pickle.dump(tracks_signatures, pkl_out)

Now we can hash the signatures to store the tracks into buckets

In [16]:
CUT_SIZE = 5  # Size of the cuts to apply to the signatures

def hash(signature_cut, max_value):
    sc = signature_cut # for conciseness
    s = 87*sc[0]+23*sc[1]+125*sc[2]+4*sc[3]+111*sc[4] # Values from the book, plus a random one
    return s % max_value

max_value_for_hash = 433 # Next prime after 431, length of the multi-hot vectors

buckets = {}

for track_name, signature in tracks_signatures.items():
    for i in range(0, len(signature), CUT_SIZE):
        cut = signature[i:i+CUT_SIZE]
        h = hash(cut, max_value_for_hash)
        if h not in buckets:
            buckets[h] = []
        buckets[h].append((track_name, signature))

with open("buckets.pkl", "wb") as buckets_out:
    pickle.dump(buckets, buckets_out)

print(len(buckets))

433


Looks like there's basically no holes in the buckets distribution, hinting that we may have a good hash function.

The next section will load and extract signatures from the query tracks, repeating most of what's been done already

In [24]:
from pathlib import Path

query_tracks = Path("datasets/ex1queries/").glob("*.wav")
query_peaks = {}
for track in query_tracks:
    _, _, _, track_peaks = load_audio_peaks(track, OFFSET, DURATION, HOP_SIZE)
    query_peaks[str(track)] = track_peaks

query_mh_peaks = {
    k: to_multi_hot(v, multihot_vectors_length) for k,v in query_peaks.items()
}

query_tracks_signatures = {}
for track_name, track_vector in query_mh_peaks.items():
    r = random.Random(123)

    v = [value for value in track_vector]
    track_signature = []
    for i in range(SIGNATURES_LENGTH):
        r.shuffle(v)
        # Append index of first non-0 value to the signature
        non_zeros = [i for i,value in enumerate(v) if value != 0]
        track_signature.append(non_zeros[0] if non_zeros else -1)

    query_tracks_signatures[track_name] = track_signature

# Here we'll store the candidate buckets for each track
query_buckets = {}

for track_name, signature in query_tracks_signatures.items():
    query_track_buckets = set()
    for i in range(0, len(signature), CUT_SIZE):
        cut = signature[i:i+CUT_SIZE]
        h = hash(cut, max_value_for_hash)
        query_track_buckets.add(h)
    query_buckets[track_name] = query_track_buckets

print(query_buckets)

{'datasets/ex1queries/track8.wav': {128, 384, 387, 5, 390, 8, 140, 401, 286, 415, 162, 295, 52, 201, 204, 344, 96, 102, 236, 372}, 'datasets/ex1queries/track9.wav': {259, 131, 389, 391, 19, 147, 426, 178, 184, 58, 77, 79, 338, 226, 227, 231, 235, 377, 124}, 'datasets/ex1queries/track10.wav': {128, 129, 3, 392, 280, 411, 284, 429, 175, 303, 181, 183, 322, 67, 71, 210, 338, 214, 364, 121}, 'datasets/ex1queries/track2.wav': {256, 6, 9, 395, 283, 28, 413, 426, 180, 53, 308, 52, 187, 321, 204, 337, 83, 355, 361, 251}, 'datasets/ex1queries/track3.wav': {130, 400, 147, 157, 285, 424, 41, 171, 428, 53, 312, 77, 208, 119, 91, 347, 351, 236, 114, 247}, 'datasets/ex1queries/track1.wav': {0, 390, 391, 139, 398, 23, 416, 37, 421, 171, 62, 326, 101, 232, 235, 242, 115, 373, 254}, 'datasets/ex1queries/track4.wav': {133, 139, 20, 154, 282, 283, 419, 43, 302, 54, 182, 81, 338, 344, 218, 95, 354, 108, 125}, 'datasets/ex1queries/track5.wav': {129, 4, 5, 397, 158, 161, 162, 420, 173, 316, 63, 201, 345, 22

Now we have to check the tracks against those in the corresponding buckets using Jaccard similarity

In [28]:
def jaccard(sig1: list, sig2: list) -> float:
    s1, s2 = set(sig1), set(sig2)
    intersection = s1.intersection(s2)
    union = s1.union(s2)
    return len(intersection)/len(union) if union else 0

scores = {} # track_name -> {candidate_name: candidate_score, ...}

for track_name, track_buckets in query_buckets.items(): # For each of the 10 query songs

    scores[track_name] = {}
    for bucket_id in track_buckets:
        for candidate_name, candidate_signature in buckets[bucket_id]:
            scores[track_name][candidate_name] = jaccard(
                query_tracks_signatures[track_name] , candidate_signature
            )

print(scores)

{'datasets/ex1queries/track8.wav': {'datasets/mp3s-32k/metallica/Metallica/10-The_God_that_Failed.wav': 0.5614035087719298, 'datasets/mp3s-32k/metallica/Ride_The_Lightning/01-Fight_Fire_With_Fire.wav': 0.5087719298245614, 'datasets/mp3s-32k/metallica/Master_of_Puppets/02-Master_of_Puppets.wav': 0.5357142857142857, 'datasets/mp3s-32k/metallica/Reload/07-Carpe_Diem_Baby.wav': 0.39436619718309857, 'datasets/mp3s-32k/metallica/Load/03-The_House_Jack_Built.wav': 0.3918918918918919, 'datasets/mp3s-32k/metallica/Load/02-2_X_4.wav': 0.5172413793103449, 'datasets/mp3s-32k/led_zeppelin/Physical_Graffiti_Disc_2_/08-Black_Country_Woman.wav': 0.5084745762711864, 'datasets/mp3s-32k/led_zeppelin/Led_Zeppelin_III/03-Celebration_Day.wav': 0.48214285714285715, 'datasets/mp3s-32k/roxette/Pearls_of_Passion_1997_Remaster_/12-So_Far_Away.wav': 0.3855421686746988, 'datasets/mp3s-32k/roxette/Have_a_Nice_Day/03-You_Can_t_Put_Your_Arms_Around_What_s_Already_Gone.wav': 0.45098039215686275, 'datasets/mp3s-32k/ste

Finally, we extract the max values for each song:

In [31]:
for track_name, candidate_scores in scores.items():

    best_match_name = "None"
    best_match_score = 0.0

    for candidate_name, candidate_score in candidate_scores.items():
        if candidate_score > best_match_score:
            best_match_score = candidate_score
            best_match_name = candidate_name

    print(f"Best match for {track_name} is {best_match_name} with a score of {best_match_score}")

Best match for datasets/ex1queries/track8.wav is datasets/mp3s-32k/suzanne_vega/Suzanne_Vega/04-Small_Blue_Thing.wav with a score of 0.6862745098039216
Best match for datasets/ex1queries/track9.wav is datasets/mp3s-32k/cure/Wish/10-Cut.wav with a score of 0.2619047619047619
Best match for datasets/ex1queries/track10.wav is datasets/mp3s-32k/depeche_mode/Black_Celebration/12-Breathing_in_fumes.wav with a score of 0.7169811320754716
Best match for datasets/ex1queries/track2.wav is datasets/mp3s-32k/depeche_mode/A_Broken_Frame/03-Monument.wav with a score of 0.7567567567567568
Best match for datasets/ex1queries/track3.wav is datasets/mp3s-32k/depeche_mode/Black_Celebration/08-Here_is_the_house.wav with a score of 0.5588235294117647
Best match for datasets/ex1queries/track1.wav is datasets/mp3s-32k/suzanne_vega/Suzanne_Vega/04-Small_Blue_Thing.wav with a score of 0.7333333333333333
Best match for datasets/ex1queries/track4.wav is datasets/mp3s-32k/beatles/A_Hard_Day_s_Night/05-And_I_Love_H

# Exercise 2

### 2.1 Getting your data!

First we have to merge the three datasets, let's take a look at them:

In [3]:
from hw_utils import ECHONEST_PATH, FEATURES_PATH, TRACKS_PATH
import pandas as pd

echonest, features, tracks = pd.read_csv(ECHONEST_PATH), pd.read_csv(FEATURES_PATH), pd.read_csv(TRACKS_PATH)

echonest.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_album_date,...,temporal_features_214,temporal_features_215,temporal_features_216,temporal_features_217,temporal_features_218,temporal_features_219,temporal_features_220,temporal_features_221,temporal_features_222,temporal_features_223
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,NaN,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.67696,3.61288,13.316690,262.929749
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,NaN,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.08277,6.01864,16.673548,325.581085
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,NaN,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.92649,5.86635,16.013849,356.755737
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,2008-03-11,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.41401,8.33319,21.317064,483.403809
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,NaN,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.26707,11.20267,26.454180,751.147705


In [4]:
features.head()

,track_id,chroma_cens_kurtosis_01,chroma_cens_kurtosis_02,chroma_cens_kurtosis_03,chroma_cens_kurtosis_04,chroma_cens_kurtosis_05,chroma_cens_kurtosis_06,chroma_cens_kurtosis_07,chroma_cens_kurtosis_08,chroma_cens_kurtosis_09,...,tonnetz_std_04,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


In [5]:
tracks.head()

,track_id,album_comments,album_date_created,album_date_released,album_engineer,album_favorites,album_id,album_information,album_listens,album_producer,...,track_information,track_interest,track_language_code,track_license,track_listens,track_lyricist,track_number,track_publisher,track_tags,track_title
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


All the datasets have a `track_id`, which we can use to join them!

In [10]:
merged_dataset = tracks.join(echonest, on="track_id", how="left", rsuffix="_").join(features, on="track_id", how="left", rsuffix="_")
print(f"Dataset entries: {len(merged_dataset)}")
merged_dataset.head()

Dataset entries: 106574


,track_id,album_comments,album_date_created,album_date_released,album_engineer,album_favorites,album_id,album_information,album_listens,album_producer,...,tonnetz_std_04,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,0.103717,0.025541,0.023846,41.645809,0.250488,0.018388,0.015625,0.000000,4.690596,0.014598
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,0.104279,0.016731,0.020464,-0.038451,0.234863,0.050837,0.050781,0.003418,0.297873,0.024899
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,0.131701,0.020644,0.026361,12.003415,0.472168,0.085446,0.089355,0.002441,1.567211,0.039701


In [14]:
from hw_utils import MERGED_DATASET_PATH

# Drop genres information and save the file
merged_dataset.drop(['track_genre_top','track_genres','track_genres_all'], axis=1)
merged_dataset.to_csv(MERGED_DATASET_PATH)

Now that we've got our dataset, we've got to cut down on the number of features. We may use one of several kinds of PCA algorithms, here we'll go with `sklearn`'s `Truncated Singular Value Decomposition to quickly deal with missing data (sparse inputs), which is not supported by other algorithms such as basic PCA but do occur in the dataset, as suggested [here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

In [18]:
from sklearn.decomposition import TruncatedSVD

svd_dataset_np = TruncatedSVD(30, n_iter=20).fit_transform(merged_dataset._get_numeric_data())
# Convert back to a Pandas DataFrame, save to file and show some records
svd_dataset = pd.DataFrame(svd_dataset_np, columns=[f"feature_{i}" for i in range(30)])
svd_dataset.to_csv("svd_dataset.csv")
svd_dataset.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29
0,1.720189e+04,238728.110807,-6.656156e+02,-30611.442982,2688.649129,-7578.828592,-85810.981286,2445.372917,-1500.375812,-8607.092490,...,-713.831378,-105.150307,-340.614178,11.699836,22.437831,86.399613,-665.140980,215.512805,-41.191242,11.639553
1,1.492533e+05,229733.364312,-2.157745e+04,-30302.095719,6.910932,-7599.445405,-84250.698067,-813.995064,-883.660731,-9631.535874,...,83.372125,-62.999401,-151.301759,135.600604,183.281405,-346.296443,-696.340662,-16.419148,-190.553355,74.057435
2,4.880876e+06,-77091.979697,3.000945e+06,1294.242979,-341597.336606,-8484.945633,-31933.845790,-13.835313,-12837.770265,-35487.027179,...,249.613303,-32.810361,209.050331,-73.825444,-198.053929,-86.814003,-308.002725,166.701235,1.572464,-22.456112
3,1.323757e+04,186445.547167,-7.230232e+02,21929.667475,2128.980722,-5462.111476,-64983.155794,65181.069987,-1304.019011,-6346.072721,...,-763.871044,-126.948719,586.010364,-163.282187,90.918354,94.824931,50.541924,-114.325457,-115.773858,-38.819593
4,2.764705e+06,66520.459816,1.874591e+06,-17287.810348,736841.281352,-8929.270339,-10877.008547,-284.759826,365913.722081,-66420.995629,...,279.996363,29.369139,408.567361,43.956257,205.452000,83.279271,-369.047988,-240.531989,-62.273938,-20.860263


Next step, let's run K-means on the dataset.

In [27]:
from tqdm.notebook import tqdm
import random

KMEANS_CLUSTERS = 4

def kmeans_distance(entry, cluster):
    dist = 0
    for i in range(len(entry)):
        dist += float(entry[f"feature_{i}"]-cluster[i])**2
    # Not running the sqrt since we're only interested in sorting the distances,
    # which is not affected by extracting the root
    return dist

def kmeans(dataset: pd.DataFrame, num_iterations, num_clusters):
    mins = dataset.min(axis=0)
    maxs = dataset.max(axis=0)

    # Randomly init cluster centers
    clusters = []
    for i in range(num_clusters):
        clusters.append([random.uniform(mins[i], maxs[i]) for i in range(len(mins))])

    # Run optimization iterations
    for i in range(num_iterations):
        print(f"Iteration {i+1}")

        # Cluster entries
        clustering = []
        for index in tqdm(range(len(dataset)), total=len(dataset)):
            distances = [kmeans_distance(dataset.iloc[[index]], cluster) for cluster in clusters]
            nearest_cluster = min(range(len(distances)), key=lambda idx: distances[idx]) # argmin(distances)
            clustering.append(nearest_cluster)

        # Update cluster centers
        for index in tqdm(range(len(clusters)), total=len(clusters)):
            cluster_set = [dataset.iloc[i].to_numpy() for i, cluster in zip(range(len(dataset)), clustering) if cluster == index]
            new_cluster_center = [0]*len(mins)
            for cluster_dimension in range(len(new_cluster_center)):
                for entry in cluster_set:
                    new_cluster_center[cluster_dimension] += entry[cluster_dimension]
                new_cluster_center[cluster_dimension] /= len(cluster_set)
            clusters[index] = new_cluster_center

    return clusters, clustering

clusters, clustering = kmeans(svd_dataset, 10, KMEANS_CLUSTERS)

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10


  0%|          | 0/106574 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/106574 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/106574 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/106574 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
# Save clustering
pd.DataFrame([[cluster] for cluster in clustering], columns=['Cluster']).to_csv("clusters.csv")


# Exercise 3

"You are given a list of integers, A, and another integer s. Write an algorithm that outputs all the pairs in A that equal x."

Let's start with the simplest algorithm. We simply iterate all pairs and output all those that satisfy the required condition.

```python
def f(A, s):
    result = []
    for i in range(len(A)):
        for j in range(i+1, len(A)):
            if A[i]+A[j] == s:
                result.append((A[i], A[s]))
    return result
```

For conciseness, the functions will just return the list of pairs, which can be converted to the output format through

```python
pairs = f(A,s)
print(", ".join([f"({a}, {b})" for a,b in pairs]) if pairs else "No pairs found")
```

Now, the given solution has O(1) space and O(n^2) time complexities.

If we want the output pairs to be ordered like in the example, there's little we can do to improve on the algorithm, but what if we just care about finding all valid pairs, regardless of the ordering?

Then we can we do better, here is another approach in pseudocode:

```python
def f(A, s):
    heap = build_heap(A)
    result = []
    for a in A:
        if s-a in heap:
            result.append((a, s-a))
    return result
```

In short, we build a heap from the array and for each element we use it to quickly check that the element summed to it belongs to the array.

Note that, as it's shown above, the algorithm also reports (s/2, s/2), for an even s, even when it only appears once, though this can be quickly patched by appending an occurrence counter to each node when building the heap.

This solution has O(n log n) time complexity (O(n log n) to build the heap, plus n times O(log n) accesses to check whether elements are in it) and O(n) space complexity, for the additional space in which to store the heap.

If we can sort A, then space complexity is O(1) as we can just heapify it rather than use additional memory.

Finally, we get the best result by slightly modifying the algorithm to build and use a hash table rather than a heap.

Then the complexities are O(n) in both time and space.